In [17]:
import os
import cv2
import numpy as np
import random
import sys
import testGMM
import re

In [18]:
# parameters:
# k: int, number of guassian distribution
# max_iter: int, maximum number of step in optimization
# img_name: strings, the relative path to single image, i.e. "train_images/032.jpg"
def trainGMM(K, max_iter, img_name):
    # read img
    img = cv2.imread(img_name)
    # user defined converge threshold
    tau = 0.00000000000000001

    def initialize():
        random_w = random.randint(0,img.shape[0])
        random_h = random.randint(0,img.shape[1])
        mean = [[img[random_w][random_h][0]],[img[random_w][random_h][1]],[img[random_w][random_h][2]]]
        # generate a random positive-semidefinete matrix as covariance matrix
        A = np.random.randint(1,5,(3,3))
        cov = np.dot(A, A.transpose())
        scaling = random.random() * 5.0
        return [scaling,mean,cov]

    params = [initialize() for cluster in range(K)]
    # Structure of para:
    # [[scale,mean,covariance],[scale,mean,covariance],[scale,mean,covariance]...]
    # scale is a int. Mean is a 3x1 matrix. Covariance is a 3x3 matrix

    # total_mean is the sum of all mean from different clusters
    total_mean = np.full((K,3, 3), -9999)
    prev_total_mean = np.full((K,3, 3), 9999)
    iter = 0

    # return true if MLE converges. Return false otherwise
    def check_convergence(total_mean, prev_toal_mean, tau):
        sum = 0
        for cluster in range(len(prev_toal_mean)):
            sum += np.linalg.norm(total_mean[cluster]-prev_total_mean[cluster])
        print("Check convergence difference: ", sum)
        return sum <= tau

    while iter <= max_iter and check_convergence(total_mean,prev_total_mean,tau):
        # update prev total mean
        prev_total_mean = total_mean

        # Expectation step - assign points to clusters, get cluster weight
        weights = []
        for cluster in range(K):
            # weight for a single cluster
            cluster_weights = np.zeros((img.shape[0], img.shape[1]))
            # cumulated weights add up all weights on a given pixel -- serving as denominator
            cumulated_weights = np.zeros((img.shape[0], img.shape[1]))
            cluster_scaling, cluster_mean, cluster_cov = params[cluster]

            for w in range(len(img[:, 0, 0])):
                for h in range(len(img[0, :, 0])):
                    pix = np.asmatrix([[img[w][h][0]], [img[w][h][1]], [img[w][h][2]]])
                    likelihood = testGMM.get_likelihood(pix, cluster_mean, cluster_cov)

                    ## calculate weight at position (w, h)
                    weight = cluster_scaling * likelihood
                    cumulated_weights[w][h] += weight
                    cluster_weights[w][h] = weight # probability of each pixel belonging to this cluster

            weights.append(cluster_weights) # weights for all clusters 1 to K,
            #weights[i][w][h]is the probability of the (w,h) pixel belonging to the ith cluster
        for cluster in range(K):
            weights[cluster] = np.divide(weights[cluster], cumulated_weights)

        # Maximization step - get new scaling, mean, and cov for each cluster
        for cluster in range(K):
            mean_sum = np.zeros((3,1)) # sums all weight*pixel RGB value on image
            cov_sum = np.zeros((3,3))
            sum_weights = np.sum(weights[cluster]) # sum of all the weights given a cluster
            for w in range(len(img[:, 0, 0])):
                for h in range(len(img[0, :, 0])):
                    pix = np.asmatrix([[img[w][h][0]], [img[w][h][1]], [img[w][h][2]]])
                    # calculate mean
                    mean_sum += np.multiply(weights[cluster][w][h],pix)

            new_mean = np.divide(mean_sum, sum_weights)

            for w in range(len(img[:, 0, 0])):
                for h in range(len(img[0, :, 0])):
                    pix = np.asmatrix([[img[w][h][0]], [img[w][h][1]], [img[w][h][2]]])
                    # calculate covariance
                    cov_sum += np.multiply(weights[cluster][w][h], (pix - new_mean))@((pix - new_mean).T)
            new_cov = np.divide(cov_sum, sum_weights)
            new_scaling = sum_weights / (img.shape[0]*img.shape[1])
            mean_sum += mean_sum

            total_mean[cluster] = new_mean
            print()
            # update model
            params[cluster] = (new_scaling, new_mean, new_cov)
        print("iter: ", iter)
        iter += 1
    # store weights to .npy
    if not os.path.exists("weights"):
        os.mkdir("weights")
    else:
        digit = re.findall(r'\d+\d+\d*',img_name)
        file_name = str(digit[0])+"_weight.npy"
        with open(os.path.join("weights",file_name), "wb") as f:
            np.save(f,params)

In [19]:
input_dir = "train_images"
for img_name in os.listdir(input_dir):
    img = os.path.join(input_dir, img_name)
    trainGMM(5,200,img)
    print("Finish Training for ", img)
    break

Check convergence difference:  299970.0
Finish Training for  train_images\106.jpg


E:\ProgramData\Miniconda3\envs\cmsc426\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [20]:
p = np.asarray([0,[[2],[4],[6]],[[1,2,3],[4,5,6],[7,8,9]]])
name = "train_images/68.jpg"
digit = re.findall(r'\d+\d+\d*',name)
file_name = str(digit[0])+"_weight.npy"
with open(os.path.join("weights",file_name), "wb") as f:
    np.save(f,p)

E:\ProgramData\Miniconda3\envs\cmsc426\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [21]:
with open('weights/68_weight.npy', 'rb') as f:
    a = np.load(f, allow_pickle=True)
    print(a[2])

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


In [22]:
A = [[1,2],[3,4],[5,6]]
A[2] = [7,8]
print(A)

[[1, 2], [3, 4], [7, 8]]
